In [17]:
import geopandas as gpd
import pandas as pd
import numpy as np

## Configuraciones Iniciales
seccion_2011_csv_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\09 - Estructura del Hogar\Secciones\Hogares_EstructuraDelHogar_Secciones_Censo_2011.csv"
seccion_2021_csv_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\09 - Estructura del Hogar\Secciones\Hogares_EstructuraDelHogar_Secciones_Censo_2021.csv"
atributo = "Estructura del Hogar"
valores = "Hogares"
valoresTotal = "Total_hogares_seccion"
valoresPorcentaje = "Hogares %"
cartografia_seccion_shp_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Shapefiles\da_cartografiaSeccionCensal-25830\da_cartografiaSeccionCensalPolygon.shp"
export_seccion_2011_csv_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\09 - Estructura del Hogar\Secciones\2011_Secciones_EstructuraDelHogar.csv"
export_seccion_2021_csv_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\09 - Estructura del Hogar\Secciones\2021_Secciones_EstructuraDelHogar.csv"

mapping_dict_2011_2021 = {
    'Total': 'Total',
    'Hogar con una mujer sola menor de 65 años': 'Hogar con una mujer sola menor de 65 años',
    'Hogar con un hombre solo menor de 65 años': 'Hogar con un hombre solo menor de 65 años',
    'Hogar con una mujer sola de 65 años o más': 'Hogar con una mujer sola de 65 años o más',
    'Hogar con un hombre solo de 65 años o más': 'Hogar con un hombre solo de 65 años o más',
    'Hogar con padre o madre que convive con algún hijo menor de 25 años': 'Hogar con un solo progenitor que convive con algún hijo menor de 25 años',
    'Hogar con padre o madre que convive con todos sus hijos de 25 años o más': 'Hogar con un solo progenitor que convive con todos sus hijos de 25 años o más',
    'Hogar formado por pareja sin hijos': 'Hogar formado por pareja sin hijos',
    'Hogar formado por pareja con hijos en donde algún hijo es menor de 25 años': 'Hogar formado por pareja con hijos en donde algún hijo es menor de 25 años',
    'Hogar formado por pareja con hijos en donde todos los hijos de 25 años o más': 'Hogar formado por pareja con hijos en donde todos los hijos de 25 años o más',
    'Hogar formado por pareja o padre/madre que convive con algún hijo menor de 25 años y otra(s) persona(s)': 'Hogar formado por pareja o un solo progenitor que convive con algún hijo menor de 25 años y otra(s) persona(a)',
    'Otro tipo de hogar': 'Otro tipo de hogar'
}

## Lectura inicial de datos regimen de tenencia
df_seccion_2011 =  pd.read_csv(seccion_2011_csv_path,sep=";")
df_seccion_2021 =  pd.read_csv(seccion_2021_csv_path,sep=";")
df_cartografia_distrito = gpd.read_file(cartografia_seccion_shp_path)

# Ver las columnas
#print(df_seccion_2011.columns)
#print(df_seccion_2021.columns)

# Inspeccionar los primeros registros
#print(df_seccion_2011.head())
#print(df_seccion_2021.head())

#Cambio el nombre de la columna 'Sección ' a Seccion de residencia
df_seccion_2021.rename(columns={'Distrito ': 'Distrito de residencia'}, inplace=True)
df_seccion_2021.rename(columns={'Sección ': 'Seccion de residencia'}, inplace=True)

#Se realiza una normalizacion de los csv  
df_seccion_2011_long = pd.melt(df_seccion_2011, id_vars=["Municipio de residencia","Distrito de residencia","Seccion de residencia"], var_name=atributo, value_name=valores)
#print(df_seccion_2011_long.columns)
#print(df_seccion_2011_long.head())
df_seccion_2021_long = pd.melt(df_seccion_2021, id_vars=["Municipio de residencia","Distrito de residencia","Seccion de residencia"], var_name=atributo, value_name=valores)
#print(df_seccion_2021_long.columns)
#print(df_seccion_2021_long.head())

## Eliminar filas ##
#Eliminar filas de municipio Total
df_seccion_2011_long = df_seccion_2011_long[df_seccion_2011_long['Municipio de residencia'] != 'Total']
df_seccion_2021_long = df_seccion_2021_long[df_seccion_2021_long['Municipio de residencia'] != 'Total']
#Eliminar filas de Seccion de residencia Total
df_seccion_2011_long = df_seccion_2011_long[df_seccion_2011_long['Distrito de residencia'] != 'Total']
df_seccion_2021_long = df_seccion_2021_long[df_seccion_2021_long['Distrito de residencia'] != 'Total']
#Eliminar filas de Seccion de residencia Total
df_seccion_2011_long = df_seccion_2011_long[df_seccion_2011_long['Seccion de residencia'] != 'Total']
df_seccion_2021_long = df_seccion_2021_long[df_seccion_2021_long['Seccion de residencia'] != 'Total']
#Eliminar total de atributo (Estructura del Hogar)
df_seccion_2011_long = df_seccion_2011_long[df_seccion_2011_long[atributo] != 'Total']
df_seccion_2021_long = df_seccion_2021_long[df_seccion_2021_long[atributo] != 'Total']

#Contamos el numero de valores que hay en estos casos
num_filas_en_blanco_2011 = (df_seccion_2011_long[valores].isna()).sum()
num_filas_con_asterisco_2011 = (df_seccion_2011_long[valores] == "*").sum()
num_filas_contiene_asterisco_2011 = (df_seccion_2011_long[valores].apply(str).str.contains(r'\*')).sum()
#print(num_filas_en_blanco_2011)#0
#print(num_filas_con_asterisco_2011)#3790
#print(num_filas_contiene_asterisco_2011)#3790
num_filas_en_blanco_2021 = (df_seccion_2021_long[valores].isna()).sum()
num_filas_con_asterisco_2021 = (df_seccion_2021_long[valores] == "*").sum()
num_filas_contiene_asterisco_2021 = (df_seccion_2021_long[valores].apply(str).str.contains(r'\*')).sum()
#print(num_filas_en_blanco_2021)#0
#print(num_filas_con_asterisco_2021)#0
#print(num_filas_contiene_asterisco_2021)#0

## Añadir columnas ##
#Añadir columna NUMSECCENS con el NUM de la seccion
df_seccion_2011_long["NUMSECCENS"] = df_seccion_2011_long["Seccion de residencia"].str[-5:].astype(float)
df_seccion_2021_long["NUMSECCENS"] = df_seccion_2021_long["Seccion de residencia"].str[-5:].astype(float)

## Gestion de Nulos y valores extraños ##
# Limpiar los datos: eliminar el asterisco al final, pero dejar intacto el '*'
df_seccion_2011_long[valores] = df_seccion_2011_long[valores].replace(r'(\d+)\*$', r'\1', regex=True)
df_seccion_2021_long[valores] = df_seccion_2021_long[valores].replace(r'(\d+)\*$', r'\1', regex=True)
#Sustituimos los valores * por NA
df_seccion_2011_long[valores] = df_seccion_2011_long[valores].replace('*', np.nan)
df_seccion_2021_long[valores] = df_seccion_2021_long[valores].replace('*', np.nan)
#Columna Personas como float
df_seccion_2011_long[valores] = df_seccion_2011_long[valores].astype(float)
df_seccion_2021_long[valores] = df_seccion_2021_long[valores].astype(float)

#Contamos el numero de valores que hay en estos casos
num_filas_en_blanco_2011 = (df_seccion_2011_long[valores].isna()).sum()
num_filas_con_asterisco_2011 = (df_seccion_2011_long[valores] == "*").sum()
num_filas_contiene_asterisco_2011 = (df_seccion_2011_long[valores].apply(str).str.contains(r'\*')).sum()
#print(num_filas_en_blanco_2011)#3790
#print(num_filas_con_asterisco_2011)#0
#print(num_filas_contiene_asterisco_2011)#0
num_filas_en_blanco_2021 = (df_seccion_2021_long[valores].isna()).sum()
num_filas_con_asterisco_2021 = (df_seccion_2021_long[valores] == "*").sum()
num_filas_contiene_asterisco_2021 = (df_seccion_2021_long[valores].apply(str).str.contains(r'\*')).sum()
#print(num_filas_en_blanco_2021)#0
#print(num_filas_con_asterisco_2021)#0
#print(num_filas_contiene_asterisco_2021)#0

#Hay valores vacios que sabemos eran * y segun el INE su valor esta entre 1-5. Usaremos el 3 como valor arbitrario (media)
# Calcular la desviación estándar antes de la imputación
desviacion_imputada_2011= df_seccion_2011_long.groupby('Seccion de residencia')[valores].std()
#print("Desviación estándar por seccion antes de imputar 3:", desviacion_imputada_2011)
#Vamos a imputar un 3 y ver si hay mucha desviacion
df_seccion_2011_long[valores] = df_seccion_2011_long[valores].replace(np.nan, 3)
df_seccion_2021_long[valores] = df_seccion_2021_long[valores].replace(np.nan, 3)
# Calcular la desviación estándar despues de la imputación
desviacion_imputada_2011= df_seccion_2011_long.groupby('Seccion de residencia')[valores].std()
#print("Desviación estándar por seccion despues de imputar 3:", desviacion_imputada_2011)

##Mapear las categorias de 2011 a 2021 para que sean iguales en ambos censos
df_seccion_2011_long[atributo] = df_seccion_2011_long[atributo].map(mapping_dict_2011_2021)

##Calcular el total de personas por distrito y el porcentaje 
df_seccion_2011_long[valoresTotal] = df_seccion_2011_long.groupby('Seccion de residencia')[valores].transform('sum')
df_seccion_2021_long[valoresTotal] = df_seccion_2021_long.groupby('Seccion de residencia')[valores].transform('sum')
#Calcular porcentaje sobre el total
df_seccion_2011_long[valoresPorcentaje] = (df_seccion_2011_long[valores] / df_seccion_2011_long[valoresTotal])
df_seccion_2021_long[valoresPorcentaje] = (df_seccion_2021_long[valores] / df_seccion_2021_long[valoresTotal])
#print(df_seccion_2011_long[df_seccion_2011_long['NUMSECCENS'] == 5002])
#print(df_seccion_2011_long[df_seccion_2011_long['NUMSECCENS'] == 2017])
#print(df_seccion_2011_long[df_seccion_2011_long['NUMSECCENS'] == 2026])
#print(df_seccion_2021_long[df_seccion_2021_long['NUMSECCENS'] == 5002])

#Pivotar la tabla de nuevo para crear variables, una por categoria con el % ; una fila por distrito
df_seccion_2011_long  = df_seccion_2011_long.pivot(index='NUMSECCENS', columns=atributo, values=valoresPorcentaje)
df_seccion_2021_long  = df_seccion_2021_long.pivot(index='NUMSECCENS', columns=atributo, values=valoresPorcentaje)

print(df_seccion_2011_long)
print(df_seccion_2011_long.count())#428
print(df_seccion_2021_long)
print(df_seccion_2021_long.count())#185


Estructura del Hogar  Hogar con un hombre solo de 65 años o más  \
NUMSECCENS                                                        
1001.0                                                 0.005137   
1002.0                                                 0.009317   
1003.0                                                 0.006211   
1004.0                                                 0.008671   
2001.0                                                 0.026087   
...                                                         ...   
11006.0                                                0.004992   
11007.0                                                0.004161   
11008.0                                                0.003484   
11009.0                                                0.014151   
11010.0                                                0.007614   

Estructura del Hogar  Hogar con un hombre solo menor de 65 años  \
NUMSECCENS                                                   

In [19]:
##Export 11 variables
df_seccion_2011_long.to_csv(export_seccion_2011_csv_path,index=True,sep=";")
df_seccion_2021_long.to_csv(export_seccion_2021_csv_path,index=True,sep=";")